In [1]:
pip install torchvision


  Obtaining dependency information for torchvision from https://files.pythonhosted.org/packages/20/ac/ab6f42af83349e679b03c9bb18354740c6b58b17dba329fb408730230584/torchvision-0.16.0-cp311-cp311-win_amd64.whl.metadata
  Using cached torchvision-0.16.0-cp311-cp311-win_amd64.whl.metadata (6.6 kB)
Using cached torchvision-0.16.0-cp311-cp311-win_amd64.whl (1.3 MB)
Note: you may need to restart the kernel to use updated packages.


In [1]:
import xml.etree.ElementTree as ET
import cv2

ModuleNotFoundError: No module named 'cv2'

In [41]:
import PIL 
from PIL import Image
import matplotlib.pyplot as plt
import re

In [57]:
def take_revers(img_path,num):
    xml_path = img_path[:-3]+'xml'
    tree = ET.parse(xml_path)
    root = tree.getroot()
    size = root.find('size')
    
    height = int(size.find('height').text)
    width = int(size.find('width').text)
    objects = root.findall('object')
    box_arr1 = []
    box_arr2 = []

    for obj in objects:
        box = obj.find('bndbox')
        box_arr1.append([int(box.find('xmin').text),
                         int(box.find('ymin').text),
         int(box.find('xmax').text),
         int(box.find('ymax').text)])
        width_arr = [box.find('xmin'),
         box.find('xmax')]
        height_arr = [box.find('ymin'),
         box.find('ymax')]

        for i in range(2):
            width_arr[i].text = str(width-int(width_arr[i].text))
        param = width_arr[0].text
        print(param)
        width_arr[0].text=width_arr[1].text
        print (param)
        width_arr[1].text = param
        box_arr2.append([int(box.find('xmin').text),
                         int(box.find('ymin').text),
         int(box.find('xmax').text),
         int(box.find('ymax').text)])
    
    p = re.search(r'\d+\.',img_path)
    root.find('filename').text = str(num+int(img_path[p.span()[0]:p.span()[1]-1]))+'.jpg'
    tree.write(img_path[:p.span()[0]]+str(num+int(img_path[p.span()[0]:p.span()[1]-1]))+'.xml')
    
    
    im = Image.open(img_path) #
    out = im.transpose(PIL.Image.FLIP_LEFT_RIGHT) 
    out.save(img_path[:p.span()[0]]+str(num+int(img_path[p.span()[0]:p.span()[1]-1]))+'.jpg')

In [72]:
path = '/home/ivan/university/maga/hakatons/dataset/размеченные/'
num=51
for i in range(47,num):
    img_path = path+str(i+1)+'.jpg'
    print (img_path)
    take_revers(img_path,num)

/home/ivan/university/maga/hakatons/dataset/размеченные/48.jpg


FileNotFoundError: [Errno 2] No such file or directory: '/home/ivan/university/maga/hakatons/dataset/размеченные/48.xml'

In [28]:
def draw_annotations(img, bboxes, thickness=2, color=(0,255,0)):
    annotations = img.copy()
    for box in bboxes:
        tlc = (box[0], box[1])
        brc = (box[2], box[3])
        cv2.rectangle(annotations, tlc, brc, color, thickness, cv2.LINE_AA)
     
    return annotations

## Загрузка батчей для VGG 16

In [57]:
import torch
from torch.utils.data import DataLoader

In [58]:
from torchvision.datasets import ImageFolder
from torchvision.models import list_models
import torchvision

In [59]:
list_models()

['alexnet',
 'convnext_base',
 'convnext_large',
 'convnext_small',
 'convnext_tiny',
 'deeplabv3_mobilenet_v3_large',
 'deeplabv3_resnet101',
 'deeplabv3_resnet50',
 'densenet121',
 'densenet161',
 'densenet169',
 'densenet201',
 'efficientnet_b0',
 'efficientnet_b1',
 'efficientnet_b2',
 'efficientnet_b3',
 'efficientnet_b4',
 'efficientnet_b5',
 'efficientnet_b6',
 'efficientnet_b7',
 'efficientnet_v2_l',
 'efficientnet_v2_m',
 'efficientnet_v2_s',
 'fasterrcnn_mobilenet_v3_large_320_fpn',
 'fasterrcnn_mobilenet_v3_large_fpn',
 'fasterrcnn_resnet50_fpn',
 'fasterrcnn_resnet50_fpn_v2',
 'fcn_resnet101',
 'fcn_resnet50',
 'fcos_resnet50_fpn',
 'googlenet',
 'inception_v3',
 'keypointrcnn_resnet50_fpn',
 'lraspp_mobilenet_v3_large',
 'maskrcnn_resnet50_fpn',
 'maskrcnn_resnet50_fpn_v2',
 'maxvit_t',
 'mc3_18',
 'mnasnet0_5',
 'mnasnet0_75',
 'mnasnet1_0',
 'mnasnet1_3',
 'mobilenet_v2',
 'mobilenet_v3_large',
 'mobilenet_v3_small',
 'mvit_v1_b',
 'mvit_v2_s',
 'quantized_googlenet',
 '

In [60]:
from torchvision import transforms

transform = transforms.Compose(
    [
        transforms.ToTensor(),
        transforms.Resize(224),
        transforms.CenterCrop(224),
        
    ]
)

In [61]:
datasets = {
    'train': ImageFolder(
        root=r'D:\project\guns_searcher\datasetVGG\train',
        transform=transform
    ),
    'valid': ImageFolder(
        root=r'D:\project\guns_searcher\datasetVGG\test',
        transform=transform
    ),
}

#datasets['train'].class_to_idx

In [69]:
batch_size = 51
num_workers = 4
dataloaders = {
    'train': DataLoader(
        datasets['train'], 
        batch_size=batch_size,
        shuffle=True,
        num_workers=num_workers
    ),
    'valid': DataLoader(
        datasets['valid'], 
        batch_size=batch_size,
        shuffle=True,
        num_workers=num_workers
    ),
}

In [63]:
dataloaders

{'train': <torch.utils.data.dataloader.DataLoader at 0x29a51b088d0>,
 'valid': <torch.utils.data.dataloader.DataLoader at 0x29a51a69610>}

In [64]:
torch.cuda.is_available()

False

In [65]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')


In [66]:
device

device(type='cpu')

In [67]:
from torchvision import models

In [70]:
model = models.vgg16(pretrained=True, progress=False).to(device)

In [71]:
model

VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1

In [72]:
for param in model.parameters():
    param.requires_grad = False

In [73]:
model.classifier[6] = torch.nn.Linear(4096, 3).to(device)

In [74]:
model.classifier

Sequential(
  (0): Linear(in_features=25088, out_features=4096, bias=True)
  (1): ReLU(inplace=True)
  (2): Dropout(p=0.5, inplace=False)
  (3): Linear(in_features=4096, out_features=4096, bias=True)
  (4): ReLU(inplace=True)
  (5): Dropout(p=0.5, inplace=False)
  (6): Linear(in_features=4096, out_features=3, bias=True)
)

In [75]:
params_to_update = model.parameters()

print("Params to learn:")
params_to_update = []
for name,param in model.named_parameters():
    if param.requires_grad == True:
        params_to_update.append(param)

optimizer = torch.optim.Adam(params_to_update, lr=0.0001)
criterion = torch.nn.CrossEntropyLoss()

Params to learn:


In [76]:
import time
import copy

from tqdm import tqdm

def train_model(model, dataloaders, criterion, optimizer, num_epochs=10):
    since = time.time()

    val_acc_history = []

    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0

    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)

        # Each epoch has a training and validation phase
        for phase in ['train', 'valid']:
            if phase == 'train':
                model.train()  # Set model to training mode
            else:
                model.eval()   # Set model to evaluate mode

            running_loss = 0.0
            running_corrects = 0

            # Iterate over data.
            for inputs, labels in tqdm(dataloaders[phase]):
                inputs = inputs.to(device)
                labels = labels.to(device)

                # zero the parameter gradients
                optimizer.zero_grad()

                # forward
                # track history if only in train
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    loss = criterion(outputs, labels)

                    _, preds = torch.max(outputs, 1)

                    # backward + optimize only if in training phase
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                # statistics
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)
                

            epoch_loss = running_loss / len(dataloaders[phase].dataset)
            epoch_acc = running_corrects.double() / len(dataloaders[phase].dataset)

            print('{} Loss: {:.4f} Acc: {:.4f}'.format(phase, epoch_loss, epoch_acc))

            # deep copy the model
            if phase == 'valid' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())
            if phase == 'valid':
                val_acc_history.append(epoch_acc)

        print()

    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(time_elapsed // 60, time_elapsed % 60))
    print('Best val Acc: {:4f}'.format(best_acc))

    # load best model weights
    model.load_state_dict(best_model_wts)
    return model, val_acc_history

In [ ]:
model, val_acc_history = train_model(model, dataloaders, criterion, optimizer, num_epochs=10)

Epoch 0/9
----------



100%|██████████████████████████████████████████████████████████████████████████████████| 34/34 [03:58<00:00,  7.01s/it]


train Loss: 0.5446 Acc: 0.8320



100%|████████████████████████████████████████████████████████████████████████████████████| 6/6 [00:43<00:00,  7.32s/it]


valid Loss: 0.5528 Acc: 0.7881

Epoch 1/9
----------


...


100%|██████████████████████████████████████████████████████████████████████████████████| 34/34 [03:53<00:00,  6.87s/it]


train Loss: 0.1668 Acc: 0.9436



100%|████████████████████████████████████████████████████████████████████████████████████| 6/6 [00:42<00:00,  7.12s/it]


valid Loss: 0.2243 Acc: 0.9106

Epoch 8/9
----------



  0%|                                                                                           | 0/34 [00:00<?, ?it/s]

In [40]:
import matplotlib.pyplot as plt

In [41]:
val_acc_history

[tensor(0.5806, dtype=torch.float64),
 tensor(0.7634, dtype=torch.float64),
 tensor(0.8065, dtype=torch.float64),
 tensor(0.8065, dtype=torch.float64),
 tensor(0.8065, dtype=torch.float64)]

In [42]:
model.train

<bound method Module.train of VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): MaxPool2d(kernel_size=2, s

In [47]:
dataloaders['valid'].dataset

Dataset ImageFolder
    Number of datapoints: 93
    Root location: D:\project\guns_searcher\datasetVGG\test
    StandardTransform
Transform: Compose(
               ToTensor()
               Resize(size=224, interpolation=bilinear, max_size=None, antialias=warn)
               CenterCrop(size=(224, 224))
           )

In [49]:
model.

AttributeError: 'VGG' object has no attribute 'predict'

In [55]:
list(dataloaders['valid'])

[[tensor([[[[0.1373, 0.1373, 0.1373,  ..., 1.0000, 1.0000, 1.0000],
            [0.1392, 0.1392, 0.1392,  ..., 1.0000, 1.0000, 1.0000],
            [0.1431, 0.1431, 0.1431,  ..., 1.0000, 1.0000, 1.0000],
            ...,
            [0.1284, 0.1284, 0.1284,  ..., 1.0000, 1.0000, 1.0000],
            [0.1265, 0.1265, 0.1265,  ..., 1.0000, 1.0000, 1.0000],
            [0.1255, 0.1255, 0.1255,  ..., 1.0000, 1.0000, 1.0000]],
  
           [[0.1529, 0.1529, 0.1529,  ..., 1.0000, 1.0000, 1.0000],
            [0.1549, 0.1549, 0.1549,  ..., 1.0000, 1.0000, 1.0000],
            [0.1588, 0.1588, 0.1588,  ..., 1.0000, 1.0000, 1.0000],
            ...,
            [0.1520, 0.1520, 0.1520,  ..., 1.0000, 1.0000, 1.0000],
            [0.1500, 0.1500, 0.1500,  ..., 1.0000, 1.0000, 1.0000],
            [0.1490, 0.1490, 0.1490,  ..., 1.0000, 1.0000, 1.0000]],
  
           [[0.3294, 0.3294, 0.3294,  ..., 1.0000, 1.0000, 1.0000],
            [0.3314, 0.3314, 0.3314,  ..., 1.0000, 1.0000, 1.0000],
      

In [56]:
import torch
